In [ ]:
## path to datasets
datadir='../Datasets/'

In [ ]:
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

# Power grid graph

This is a test of robustness of an empirical graph that does not exhibit power law.

Network robustness is an important practical topic for power grids.

First we load data

In [ ]:
gr = ig.Graph.Read_Ncol(datadir+'GridEurope/gridkit_europe-highvoltage.edges', directed=False)
gr = gr.simplify()
## read the vertices along with some attributes
X = pd.read_csv(datadir+'GridEurope/gridkit_europe-highvoltage.vertices')
idx = [int(i) for i in gr.vs['name']]
sorterIndex = dict(zip(idx,range(len(idx))))
X['Rank'] = X['v_id'].map(sorterIndex)
X.sort_values(['Rank'], ascending=[True],inplace=True)
X.dropna(inplace=True)
gr.vs['longitude'] = list(X['lon'])
gr.vs['latitude'] = list(X['lat'])
gr.vs['type'] = list(X['typ'])
gr.vs['layout'] = [(v['longitude'],v['latitude']) for v in gr.vs()]
gr.vs['size'] = 3
gr.es['color'] = 'grey'
gr.vs['color'] = 'black'

We want to concentrate on an Iberic peninsula. Note that we select a giant component of the graph selected by latitude and longitude as e.g. nodes located on islands are isolated.

In [ ]:
## subgraph of Grid -- giant component on Iberic peninsula
V = [v for v in gr.vs() if v['latitude']>36 and v['latitude']<44 and v['longitude']>-10 and v['longitude']<4]
gr_spain = gr.subgraph(V).clusters().giant()
ly = ig.Layout(gr_spain.vs['layout'])
ly.mirror(1)
ig.plot(gr_spain, 'grid_iberic_giant.eps', layout=ly, bbox=(0,0,300,300))

We check the node count and average node degree 

In [ ]:
gr_spain.vcount()

In [ ]:
np.mean(gr_spain.degree())

This function given a graph `gr` tests removing nodes from this graph until `stop_count` nodes are left.
If `rnd` is `True` then we assume random failure model. If `rnd` is `False` then we assume targetted attack, in which sequentially a node with the highest degree in the remaining network is removed.

The function returns order parameter sequence (i.e. the fraction of nodes in the giant component of the remaining graph) and the final graph.

In [ ]:
def single_run(gr, fun, stop_count=1):
    gr = gr.copy()
    order = [1.0]
    while gr.vcount() > stop_count:
        if fun == "random":
            to_delete = random.randint(0, gr.vcount() - 1)
        elif fun == "degree":
            m = max(gr.degree())
            am = [i for i, j in enumerate(gr.degree()) if j == m]
            to_delete = random.choice(am)
        elif fun == "between":
            b = gr.betweenness()
            m = max(b)
            am = [i for i, j in enumerate(b) if j == m]
            to_delete = random.choice(am)
        else:
            raise Exception("unknown value of parameter fun")
        gr.delete_vertices(to_delete)
        order.append(gr.clusters().giant().vcount() / gr.vcount())
    return order, gr

We run both options (`rnd` equal to `True` and to `False`) 16 times and report the average result as `single_run` is randomized. Note that `rnd=True` leads to much lower variability.

In [ ]:
res_rnd = single_run(gr_spain, "random")[0]

In [ ]:
res_degree = single_run(gr_spain, "degree")[0]

In [ ]:
res_between = single_run(gr_spain, "between")[0]

Observe that order parameter:
1. Falls much faster when `rnd=True`
2. When almost all nodes are removed order parameter increases, but this is a case that is not very interesting (we are mostly interested in the left parts of the plots)

In [ ]:
plt.plot(res_rnd)
plt.plot(res_degree)
plt.plot(res_between)
plt.xlabel("# of removed vertices")
plt.ylabel("order parameter")
plt.legend(["random", "max degree attack", "max betweenness attack"])
plt.savefig('op_power_grid.eps', format='eps')

We test a single scenario under maximum degree attack when 5% of nodes are removed. Colors on the plot represent different components of the graph.

In [ ]:
_, gr_sel = single_run(gr_spain, "between", gr_spain.vcount()*0.95)

In [ ]:
ly = ig.Layout(gr_sel.vs['layout'])
ly.mirror(1)
ig.plot(gr_sel.clusters(), 'grid_iberic_5pct_attack.eps', layout=ly, bbox=(0,0,300,300))

Check the number of components

In [ ]:
len(gr_sel.clusters().sizes())

And distribution of their sizes. We can see that we have many small components.

In [ ]:
plt.hist(gr_sel.clusters().sizes(), 50);

In [ ]:
df = pd.DataFrame({'size': gr_sel.clusters().sizes()})

In [ ]:
df['size'].value_counts().sort_index()

In [ ]:
_, gr_sel = single_run(gr_spain, "degree", gr_spain.vcount()*0.95)

In [ ]:
plt.hist(gr_sel.clusters().sizes(), 50);

In [ ]:
df = pd.DataFrame({'size': gr_sel.clusters().sizes()})
df['size'].value_counts().sort_index()

This time we get more components, but the larger components are bigger.

# Power law

Here we investigate the scenario where we simulate a graph whose degree distribution follows the power law.

As the initial graph might be disconnected, note that there is a drop in order parameter when we start the process.

In all compared scenarios we generate a graph with 2000 nodes and mean degree 4 and change the power law exponent only.

In [ ]:
def gen_pl(exponent):
    g = ig.Graph.Static_Power_Law(2000, 4000, exponent)
    return (single_run(g, "random")[0], single_run(g, "degree")[0], single_run(g, "between")[0])

In [ ]:
plt.rcParams["figure.figsize"] = (15, 6)

the above computations are slow, because computing betweenness is expansive.

In [ ]:
r4, d4, b4 = gen_pl(4)
r3, d3, b3 = gen_pl(3)
r25, d25, b25 = gen_pl(2.5)
r2, d2, b2 = gen_pl(2)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
ax1.plot(r4)
ax1.plot(r3)
ax1.plot(r25)
ax1.plot(r2)
ax1.set_title("random")
ax1.legend(["4", "3", "2.5", "2"])

ax2.plot(d4)
ax2.plot(d3)
ax2.plot(d25)
ax2.plot(d2)
ax2.set_title("max degree")
ax2.legend(["4", "3", "2.5", "2"])

ax3.plot(b4)
ax3.plot(b3)
ax3.plot(b25)
ax3.plot(b2)
ax3.set_title("max betweenness")
ax3.legend(["4", "3", "2.5", "2"])

plt.savefig('op_power_law.eps', format='eps')

# Assortative

Next we turn our analysis to checking how network assortativity influences the order parameter.

In [ ]:
## Xulvi-Brunet and Sokolov algorithm with power law underlying graph
def XBS(q, assortative):
    g = ig.Graph.Static_Power_Law(2000, 4000, 3)
    g.es['touched'] = False
    ec = g.ecount()
    while True:
        re = np.random.choice(ec, 2, replace=False)
        nodes = list(g.es[re[0]].tuple+g.es[re[1]].tuple)
        if len(set(nodes))==4:
            ## with proba q, wire w.r.t. assortativity, else randomly
            if np.random.random()<q:
                idx = np.argsort(g.degree(nodes))
                if assortative:
                    e1 = (nodes[idx[0]],nodes[idx[1]])
                    e2 = (nodes[idx[2]],nodes[idx[3]])
                else:
                    e1 = (nodes[idx[0]],nodes[idx[3]])
                    e2 = (nodes[idx[1]],nodes[idx[2]])
            else:
                np.random.shuffle(nodes)
                e1 = (nodes[0],nodes[1])
                e2 = (nodes[2],nodes[3])
            if g.get_eid(e1[0], e1[1], directed=False, error=False)+\
               g.get_eid(e2[0], e2[1], directed=False, error=False) == -2:
                    g.delete_edges(re)
                    g.add_edge(e1[0],e1[1],touched=True)
                    g.add_edge(e2[0],e2[1],touched=True)
            else:
                g.es[re[0]]['touched']=True
                g.es[re[1]]['touched']=True
        if sum(g.es['touched']) == g.ecount():
            break
    return (single_run(g, "random")[0], single_run(g, "degree")[0], single_run(g, "between")[0])

In the scenarios we again make the graphs to have a comparable number of nodes and average degree to grid network.

Positive assortativity scenario

In [ ]:
ra, da, ba = XBS(3/4, True)
rd, dd, bd = XBS(3/4, False)
r0, d0, b0 = XBS(0, True)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
ax1.plot(ra)
ax1.plot(rd)
ax1.plot(r0)
ax1.set_title("random")
ax1.legend(["assortative", "dissortative", "zero"])

ax2.plot(da)
ax2.plot(dd)
ax2.plot(d0)
ax2.set_title("max degree")
ax2.legend(["assortative", "dissortative", "zero"])

ax3.plot(ba)
ax3.plot(bd)
ax3.plot(b0)
ax3.set_title("max betweenness")
ax3.legend(["assortative", "dissortative", "zero"])

plt.savefig('op_assortative.eps', format='eps')

Negative assortivity scenario

No assortivity scenario

We can observe that for random failure model the higher assortativity makes the decreas of order parameter more evenly distributed.

For maximum degree attack scenario we note the reverse - lower assortativity leads to more evenly distributed decerase of order parameter.

# Communities

The last experiment we perform is with a graph that contains communities. We used ABCD graph generator with $\xi\in\{0.1,0.4,0.7,1.0\}$.

Again we tune the graphs to roughly follow the node count and average node degree of a grid graph.

In [ ]:
def read_graph(name):
    g = ig.Graph.Read_Ncol(datadir+'ABCD/edge'+name+'.dat',directed=False)
    print("node count: ", g.vcount())
    print("mean degree:", np.mean(g.degree()))
    return single_run(g, "random")[0], single_run(g, "degree")[0], single_run(g, "between")[0]

As you can see - in this case we do not see a significant influence of $\xi$ parameter on the results.

In [ ]:
r01, d01, b01 = read_graph('01')
r04, d04, b04 = read_graph('04')
r07, d07, b07 = read_graph('07')
r10, d10, b10 = read_graph('10')

fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
ax1.plot(r01)
ax1.plot(r04)
ax1.plot(r07)
ax1.plot(r10)
ax1.set_title("random")
ax1.legend(["0.1", "0.4", "0.7", "1.0"])

ax2.plot(d01)
ax2.plot(d04)
ax2.plot(d07)
ax2.plot(d10)
ax2.set_title("max degree")
ax2.legend(["0.1", "0.4", "0.7", "1.0"])

ax3.plot(b01)
ax3.plot(b04)
ax3.plot(b07)
ax3.plot(b10)
ax3.set_title("max betweenness")
ax3.legend(["0.1", "0.4", "0.7", "1.0"])

plt.savefig('op_communities.eps', format='eps')

In [ ]:
fig, (ax2, ax3) = plt.subplots(1, 2)
ax2.plot(d01[:300])
ax2.plot(d04[:300])
ax2.plot(d07[:300])
ax2.plot(d10[:300])
ax2.set_title("max degree")
ax2.legend(["0.1", "0.4", "0.7", "1.0"])

ax3.plot(b01[:300])
ax3.plot(b04[:300])
ax3.plot(b07[:300])
ax3.plot(b10[:300])
ax3.set_title("max betweenness")
ax3.legend(["0.1", "0.4", "0.7", "1.0"])